In [ ]:
def convert_txt_to_json(input_path, output_path):
    import json
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for id, sample in enumerate(f.read().strip().split('\n\n')):
            tokens, tags = [], []
            for line in sample.strip().split('\n'):
                if line:
                    token, tag = line.split()
                    tokens.append(token)
                    tags.append(tag)
            data.append({'id': id,'tokens': tokens, 'ner_tags': tags})
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


convert_txt_to_json('../data/medical/dev.txt', '../data/medical/dev_tese.json')

In [1]:
import os
import swanlab


swanlab.login(api_key="EOQCnWdMl5RFgMpUiQpTd", save=True)
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

/home/aligames/anaconda3/envs/stage3/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [2]:
from datasets import load_dataset, load_from_disk

raw_dataset = load_from_disk('medical-ner')
raw_dataset

/home/aligames/anaconda3/envs/stage3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5259
    })
    dev: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 657
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 658
    })
})

In [3]:
raw_dataset['dev'].features

{'id': Value(dtype='int32', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-临床表现', 'I-临床表现', 'B-中医治疗', 'I-中医治疗', 'B-西医诊断', 'I-西医诊断', 'B-方剂', 'I-方剂', 'B-中药', 'I-中药', 'B-中医诊断', 'I-中医诊断', 'B-西医治疗', 'I-西医治疗', 'B-中医证候', 'I-中医证候', 'B-中医治则', 'I-中医治则', 'B-其他治疗', 'I-其他治疗'], id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [4]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification


checkpoint = 'Qwen/Qwen2.5-3B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForTokenClassification.from_pretrained(checkpoint, num_labels=len(raw_dataset['dev'].features['ner_tags'].feature.names))

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.47it/s]
Some weights of Qwen2ForTokenClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-3B-Instruct and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# 获取可训练参数
trainable_params = [p for p in model.parameters() if p.requires_grad]

# 打印可训练参数的数量
print("Number of trainable parameters:", sum(p.numel() for p in trainable_params))

Number of trainable parameters: 3085981717


In [7]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(task_type=TaskType.TOKEN_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model=model, peft_config=peft_config).to(device)
model.print_trainable_parameters()

trainable params: 1,886,229 || all params: 3,087,867,946 || trainable%: 0.0611


In [8]:
def process_func(examples):
    tokenized_examples = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True, max_length=512)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_examples.word_ids(batch_index=i)
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
        labels.append(label_ids)
    tokenized_examples['labels'] = labels
    return tokenized_examples

In [9]:
tokenized_dataset = raw_dataset.map(process_func, batched=True)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5259
    })
    dev: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 657
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 658
    })
})

In [10]:
import evaluate

seqeval_matric = evaluate.load('seqeval')

In [11]:
names = tokenized_dataset['dev'].features['ner_tags'].feature.names

In [12]:
import numpy as np
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

# 假设 label_list 是从模型中获取的所有标签列表，例如 ['O', 'B-PER', 'I-PER', ...]
label_list = names  # 你的标签列表

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=-1)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # 使用 seqeval 的 classification_report 获取详细分类结果
    report = classification_report(true_labels, true_predictions, mode='strict', scheme=IOB2)

    # 解析 report 成 dict 形式
    metric_dict = {}
    lines = report.strip().split('\n')
    for line in lines[2:-3]:  # 跳过头部和底部 summary 行
        parts = line.strip().split()
        if len(parts) == 5:
            cls, precision, recall, f1, support = parts
            metric_dict[f"f1_{cls}"] = float(f1)

    # 添加 overall 指标
    from seqeval.metrics import precision_score, recall_score, f1_score
    metric_dict["precision"] = precision_score(true_labels, true_predictions, mode='strict', scheme=IOB2)
    metric_dict["recall"] = recall_score(true_labels, true_predictions, mode='strict', scheme=IOB2)
    metric_dict["f1"] = f1_score(true_labels, true_predictions, mode='strict', scheme=IOB2)

    return metric_dict

In [13]:
from transformers import TrainingArguments

args = TrainingArguments(
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    # gradient_accumulation_steps=4,
    num_train_epochs=20,
    weight_decay=0.01,
    remove_unused_columns=True,
    output_dir='../output',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    bf16=True,
    report_to="swanlab"
)

In [14]:
from transformers import Trainer, DataCollatorForTokenClassification

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset['train'].shuffle(42),
    eval_dataset=tokenized_dataset['dev'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

/tmp/ipykernel_173264/3792506280.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForTokenClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
trainer.train()

/home/aligames/anaconda3/envs/stage3/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

swanlab: Tracking run with swanlab version 0.6.5

swanlab: Run data will be saved locally in 
/home/aligames/project/stage3/src/swanlog/run-20250708_174531-9dzxfu18h9g76p2zowoci

swanlab: 👋 Hi Wolffy,welcome to swanlab!

swanlab: Syncing run ../output to the cloud

swanlab: 🏠 View project at https://swanlab.cn/@Wolffy/src

swanlab: 🚀 View run at https://swanlab.cn/@Wolffy/src/runs/9dzxfu18h9g76p2zowoci

Epoch,Training Loss,Validation Loss,F1 中医治则,F1 中医治疗,F1 中医证候,F1 中医诊断,F1 中药,F1 临床表现,F1 其他治疗,F1 方剂,F1 西医治疗,F1 西医诊断,Precision,Recall,F1
1,0.509400,0.438415,0.000000,0.130000,0.620000,0.000000,0.320000,0.340000,0.000000,0.000000,0.000000,0.630000,0.464351,0.313387,0.374217
2,0.366600,0.355293,0.080000,0.170000,0.700000,0.000000,0.710000,0.490000,0.000000,0.060000,0.030000,0.690000,0.592914,0.505861,0.545939
3,0.313500,0.330534,0.000000,0.190000,0.700000,0.230000,0.710000,0.560000,0.000000,0.060000,0.170000,0.740000,0.598945,0.560148,0.578897
4,0.358300,0.318696,0.280000,0.230000,0.750000,0.260000,0.710000,0.580000,0.000000,0.070000,0.290000,0.760000,0.612799,0.584824,0.598485
5,0.261000,0.312240,0.220000,0.300000,0.760000,0.360000,0.700000,0.560000,0.200000,0.130000,0.350000,0.770000,0.609335,0.587909,0.598430
6,0.278000,0.309130,0.160000,0.300000,0.750000,0.330000,0.740000,0.580000,0.400000,0.210000,0.350000,0.780000,0.624922,0.615669,0.620261
7,0.253500,0.308608,0.070000,0.300000,0.740000,0.360000,0.650000,0.560000,0.000000,0.070000,0.350000,0.780000,0.628963,0.538556,0.580259
8,0.253000,0.305247,0.210000,0.340000,0.730000,0.350000,0.690000,0.570000,0.220000,0.160000,0.430000,0.770000,0.628743,0.582973,0.604994
9,0.250100,0.308237,0.210000,0.370000,0.780000,0.300000,0.710000,0.580000,0.000000,0.180000,0.400000,0.780000,0.625712,0.609500,0.617500
10,0.225400,0.310467,0.210000,0.350000,0.740000,0.390000,0.690000,0.570000,0.000000,0.130000,0.350000,0.760000,0.621639,0.584824,0.602670


/home/aligames/anaconda3/envs/stage3/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aligames/anaconda3/envs/stage3/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aligames/anaconda3/envs/stage3/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=13160, training_loss=0.31382910982453716, metrics={'train_runtime': 2451.8346, 'train_samples_per_second': 42.898, 'train_steps_per_second': 5.367, 'total_flos': 1.3880805583195034e+17, 'train_loss': 0.31382910982453716, 'epoch': 20.0})

In [6]:
model

Qwen2ForTokenClassification(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((2048,), eps=1e-06)
    (r